In [ ]:
!pip install ucimlrepo

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ucimlrepo import fetch_ucirepo

# --- 1. Chargement et inspection des données ---
print("--- 1. Chargement du jeu de données Online Retail ---")
# fetch dataset
online_retail = fetch_ucirepo(id=352)

# data (as pandas dataframes)
df = online_retail.data.features.copy() # X
df['Target'] = online_retail.data.targets # y (Optional: Add the Target/Description if it exists)
# Note: The 'Online Retail' dataset is often used for Association Rule Mining (like Apriori) or RFM analysis,
# so we'll primarily focus on the features.

# Affichage des premières lignes et des types de données
print("\n📌 Aperçu des 5 premières lignes du DataFrame:")
print(df.head())
print("\n📌 Informations sur les types de données et les valeurs non nulles:")
df.info()

# --- 2. Nettoyage et préparation des données ---

# Convertir InvoiceDate en datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Supprimer les lignes avec des valeurs manquantes dans StockCode et CustomerID
# C'est crucial pour l'analyse client (RFM)
df.dropna(subset=['StockCode', 'CustomerID'], inplace=True)

# S'assurer que Quantity et UnitPrice sont positifs pour l'analyse des ventes
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]

# Calculer le montant total de la transaction
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

print("\n--- 2. Résumé après nettoyage et calcul de TotalPrice ---")
print(f"Nombre de lignes après nettoyage : {len(df)}")
print(df[['Quantity', 'UnitPrice', 'TotalPrice']].describe().T)

# --- 3. Analyse et Visualisations Clés ---

# Définir le style des graphiques
sns.set_style("whitegrid")
plt.figure(figsize=(18, 15))

# --- Graphique 1: Distribution des prix unitaires (log-transformé) ---
plt.subplot(3, 2, 1)
# Utiliser log1p pour gérer la forte asymétrie
sns.histplot(np.log1p(df['UnitPrice']), bins=50, kde=True, color='skyblue')
plt.title('Distribution du Prix Unitaire (Transformation Logarithmique)')
plt.xlabel('log(1 + UnitPrice)')
plt.ylabel('Fréquence')

# --- Graphique 2: Top 10 des pays par nombre de transactions ---
plt.subplot(3, 2, 2)
top_countries = df['Country'].value_counts().nlargest(10)
sns.barplot(x=top_countries.index, y=top_countries.values, palette='viridis')
plt.title('Top 10 des Pays par Nombre de Transactions')
plt.xlabel('Pays')
plt.ylabel('Nombre de Transactions')
plt.xticks(rotation=45, ha='right')

# --- Graphique 3: Montant total des ventes au fil du temps (par mois) ---
plt.subplot(3, 1, 2)
df['InvoiceMonth'] = df['InvoiceDate'].dt.to_period('M')
monthly_sales = df.groupby('InvoiceMonth')['TotalPrice'].sum()
monthly_sales.index = monthly_sales.index.astype(str) # Convert PeriodIndex to string for plotting
monthly_sales.plot(kind='line', marker='o', color='forestgreen', linewidth=2)
plt.title('Tendance des Ventes Totales Mensuelles')
plt.xlabel('Mois')
plt.ylabel('Ventes Totales (en millions)')
plt.ticklabel_format(style='plain', axis='y') # Éviter la notation scientifique sur l'axe y
# Re-formater les étiquettes de l'axe y pour une meilleure lisibilité (par ex. en millions)
# Les données sont en livres sterling, donc diviser par 1,000,000 pour les millions
y_labels = ['{:,.0f}M'.format(y/1000000) for y in plt.gca().get_yticks()]
plt.gca().set_yticklabels(y_labels)

# --- Graphique 4: Top 10 des produits (StockCode) par revenu total ---
plt.subplot(3, 2, 5)
top_products = df.groupby('StockCode')['TotalPrice'].sum().nlargest(10).sort_values(ascending=False)
sns.barplot(x=top_products.index, y=top_products.values, palette='plasma')
plt.title('Top 10 des Codes Stock (Produits) par Revenu Total')
plt.xlabel('Code Stock')
plt.ylabel('Revenu Total')
plt.xticks(rotation=45, ha='right')

# --- Graphique 5: Distribution des quantités vendues ---
plt.subplot(3, 2, 6)
# Limiter l'axe x pour une meilleure visualisation (ignorer les très grandes quantités)
sns.boxplot(y=df[df['Quantity'] < 100]['Quantity'], color='coral')
plt.title('Distribution des Quantités (Quantité < 100)')
plt.ylabel('Quantité')

plt.tight_layout(pad=3.0)
plt.show()

# --- 4. Informations Complémentaires Clés ---
print("\n--- 4. Informations Clés (Post-Nettoyage) ---")
print(f"Période de temps couverte : {df['InvoiceDate'].min().strftime('%Y-%m-%d')} à {df['InvoiceDate'].max().strftime('%Y-%m-%d')}")
print(f"Nombre total de clients uniques : {df['CustomerID'].nunique()}")
print(f"Revenu total (après nettoyage) : £{df['TotalPrice'].sum():,.2f}")